In [22]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:

# True: not answer all question，False: answer all questions mandatorily。
answer_all = True
# load Bert
model_checkpoint = "distilbert-base-uncased"

batch_size = 16


In [3]:
# qa pairs
import pandas as pd
from datasets import Dataset, DatasetDict
with open('../testing_QA/reformatting_testing_qa.csv','r',encoding='utf-8') as f:
    df_c = pd.read_csv(f)


In [5]:
df_c

,question,answer,start_point,article_title,PMID,author,context,article_date
0,Who are the authors of the article on the disc...,"Nader AM', 'Jelenic P', 'Soulieres I'","Author: ['Nader AM', 'Jelenic P', 'Soulieres I']",Discrepancy between WISC-III and WISC-IV Cogni...,26673881,"Nader AM, Jelenic P, Soulieres I",The cognitive profile and measured intellectua...,2015/12/18
1,What is the focus of the cognitive profile exa...,Examine the cognitive profile on the Wechsler ...,Abstract:The cognitive profile and measured in...,Discrepancy between WISC-III and WISC-IV Cogni...,26673881,"Nader AM, Jelenic P, Soulieres I",The cognitive profile and measured intellectua...,2015/12/18
2,How many children with autism were part of the...,51,The third aim was to examine the impact of the...,Discrepancy between WISC-III and WISC-IV Cogni...,26673881,"Nader AM, Jelenic P, Soulieres I",The cognitive profile and measured intellectua...,2015/12/18
3,What percentage increase was observed in the d...,nearly tripled,Autistic children scored significantly higher ...,Discrepancy between WISC-III and WISC-IV Cogni...,26673881,"Nader AM, Jelenic P, Soulieres I",The cognitive profile and measured intellectua...,2015/12/18
4,What are the two main perspectives focused on ...,Two different perspectives are the main focus ...,69,Systems Medicine in Oncology: Signaling Networ...,26677185,"Parodi S, Riccardi G, Castagnino N, Tortolina ...",Two different perspectives are the main focus ...,2015/12/18
...,...,...,...,...,...,...,...,...
74,What signs of manipulation were found in the c...,"addition of glue on the boxes and the vials, d...",1171,Counterfeit analysis strategy illustrated by a...,26692552,"Degardin K, Roggo Y",Medicine counterfeiting is a current problem t...,2015/12/23
75,Who are the authors of the paper on nature-ins...,"Siddique N', 'Adeli H","Author: ['Siddique N', 'Adeli H']",Nature Inspired Computing: An Overview and Som...,26693257,"Siddique N, Adeli H",This paper presents an overview of significant...,2015/12/23
76,What is the focus of the paper on nature-inspi...,significant advances made in the emerging fiel...,This paper presents an overview of significant...,Nature Inspired Computing: An Overview and Som...,26693257,"Siddique N, Adeli H",This paper presents an overview of significant...,2015/12/23
77,What has paralleled the development of 'nature...,the emergence of the field of computational in...,A parallel development in the past two decades...,Nature Inspired Computing: An Overview and Som...,26693257,"Siddique N, Adeli H",This paper presents an overview of significant...,2015/12/23


In [3]:
pd.options.mode.copy_on_write = True

In [7]:
from copy import deepcopy
df_latest = deepcopy(df_c)

col_name=df_latest.columns.tolist() 
col_name.insert(0, 'id')

df_latest = df_latest.reindex(columns=col_name)  

for i in range(len(df_c['answer'])):
    new_context = f"{df_c['article_title'][i]}\n{df_c['author'][i]}\n{df_c['context'][i]}\n{df_c['article_date'][i]}\n{df_c['PMID'][i]}"
    if not df_c['start_point'][i].isnumeric():
        start_point = new_context.index(' '.join(df_c['start_point'][i].split()[0:2]))
        df_latest.replace({"start_point": {
                df_c['start_point'][i]: str([start_point])
            }}, inplace=True)
        temp_answer = {'text': [df_c['answer'][i]], 'answer_start': [start_point]}
    else:
        temp_answer = {'text': [df_c['answer'][i]], 'answer_start': [df_c['start_point'][i]]}
    df_latest.replace({"answer": {
                df_c['answer'][i]: str(temp_answer)
            }}, inplace=True)

    df_latest.replace({"context": {
                df_c['context'][i]: new_context
            }}, inplace=True)
    
    df_latest.at[i, 'id']= str(int(i))

ValueError: substring not found

In [7]:
df_latest

,id,question,answer,start_point,article_title,PMID,author,context,article_date
0,0,What is the purpose of the prospective study m...,{'text': ['To define the very early neuropsych...,259,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",Neuropsychological profile in new-onset benign...,2015/12/16
1,1,How many children with BECTS were enrolled in ...,{'text': ['Fifteen school-aged children with B...,438,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",Neuropsychological profile in new-onset benign...,2015/12/16
2,2,What neuropsychological domains showed signifi...,"{'text': ['Language, executive functions, and ...",1041,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",Neuropsychological profile in new-onset benign...,2015/12/16
3,3,What implications are drawn from the results r...,{'text': ['Children with new-onset BECTS may d...,2288,Neuropsychological profile in new-onset benign...,26667848,"Filippini M, Ardu E, Stefanelli S, Boni A, Gob...",Neuropsychological profile in new-onset benign...,2015/12/16
4,4,What is the London Child Health and Developmen...,{'text': ['The London Child Health and Develop...,139,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",Toward earlier identification and preventative...,2015/12/17
5,5,What is the purpose of the London Child Health...,{'text': ['The purpose of the London Child Hea...,325,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",Toward earlier identification and preventative...,2015/12/17
6,6,What does the London Child Health and Developm...,{'text': ['The London Child Health and Develop...,621,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",Toward earlier identification and preventative...,2015/12/17
7,7,What are the findings of the London Child Heal...,{'text': ['The findings of the London Child He...,900,Toward earlier identification and preventative...,26670311,"Laurens KR, Cullen AE",Toward earlier identification and preventative...,2015/12/17
8,8,What is the core hypothesis in developmental t...,{'text': ['genetic influences on intelligence ...,276,Large Cross-National Differences in Gene x Soc...,26671911,"Tucker-Drob EM, Bates TC",Large Cross-National Differences in Gene x Soc...,2015/12/17
9,9,What did the meta-analysis by Tucker-Drob and ...,{'text': ['Gene x SES interaction on intellige...,287,Large Cross-National Differences in Gene x Soc...,26671911,"Tucker-Drob EM, Bates TC",Large Cross-National Differences in Gene x Soc...,2015/12/17


In [8]:
# from datasets import DatasetDict
# dataset = DatasetDict.from_csv(
#         path_or_paths{'train': '../testing_QA/reformatting_testing_qa.csv'},
#         features: Optional[Features] = None,
#         cache_dir: str = None,
#         keep_in_memory: bool = False,
#         **kwargs,
#     )
# or we can do
# load_dataset("csv", data_files="my_file.csv")

In [9]:
training_df = df_latest.sample(frac=0.8, axis=0, replace=False, random_state=0)

In [10]:
testing_df = df_latest[~df_latest.index.isin(training_df.index)]

In [11]:
# construct datasets
training_data = Dataset.from_pandas(training_df, preserve_index=False)

In [12]:
training_data

Dataset({
    features: ['id', 'question', 'answer', 'start_point', 'article_title', 'PMID', 'author', 'context', 'article_date'],
    num_rows: 16
})

In [13]:
testing_data = Dataset.from_pandas(testing_df, preserve_index=False)

In [14]:
dataset = DatasetDict()
dataset['train'] = training_data
dataset['validation'] = testing_data

In [8]:
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
import torch
import collections
from tqdm.auto import tqdm
import ast

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
max_length = 384 # max length of input feature
doc_stride = 128 # repeat tokens number between 2 slices

In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
tokenizer("What is your name?")

{'input_ids': [101, 2054, 2003, 2115, 2171, 1029, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [13]:
for i, example in enumerate(training_data):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = training_data[i]

In [21]:
len(tokenizer(example["question"], example["context"])["input_ids"])

416

In [22]:
tokenized_example = tokenizer(
    example["question"], # question
    example["context"],  # abstract
    max_length=max_length, 
    truncation="only_second", # slice only in the context
    return_overflowing_tokens=True,  # flag the slicing if exceed max length
    return_offsets_mapping=True,
    stride=doc_stride # slice step size
)

In [23]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 181]

In [24]:
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (5, 10), (11, 15), (16, 20), (21, 23), (24, 27), (28, 36), (37, 39), (40, 46), (47, 53), (54, 61), (61, 62), (63, 76), (77, 83), (84, 87), (88, 97), (98, 110), (110, 111), (0, 0), (0, 13), (14, 20), (21, 24), (25, 34), (35, 47), (48, 50), (51, 56), (57, 63), (64, 70), (71, 78), (78, 79), (80, 82), (82, 83), (83, 86), (86, 89), (90, 91), (91, 92), (93, 97), (97, 100), (101, 102), (103, 107), (108, 114), (115, 116), (116, 119), (120, 128), (129, 135), (135, 136), (137, 140), (141, 147), (147, 154), (154, 156), (157, 166), (167, 169), (170, 171), (172, 176), (177, 185), (185, 186), (187, 191), (192, 194), (194, 195), (196, 200), (201, 204), (205, 209), (210, 215), (216, 222), (223, 225), (226, 232), (233, 244), (245, 253), (253, 254), (255, 260), (261, 264), (265, 271), (272, 274), (275, 280), (281, 289), (290, 303), (304, 316), (317, 320), (321, 324), (325, 333), (334, 336), (337, 342), (343, 348), (349, 354), (355, 365), (365, 366), (367, 384), (385, 392), (393, 397), (

In [25]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [14]:
pad_on_right = tokenizer.padding_side == "right" #context on the right

In [15]:
def prepare_train_features(examples):
    # truncation and padding
    tokenized_examples = tokenizer(
        examples['question' if pad_on_right else "context"],
        examples['context' if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    offset_mapping = tokenized_examples.pop("offset_mapping")

    # tag data again
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # each slice processing
        # no answer sample tagged to CLS
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # distinguish question and context
        sequence_ids = tokenized_examples.sequence_ids(i)

        # original example index
        sample_index = sample_mapping[i]
        answers = examples["answer"][sample_index]
        # no answer, CLS location as answer
        if type(answers)==str:
            answers = ast.literal_eval(answers)
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # character level's start/end
            start_char = int(answers["answer_start"][0])
            end_char = start_char + len(answers["text"][0])

            # token level's index start
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # token level's index end
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # checking if exceeding length，if so, CLS index as tag
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Note: we could go after the last offset if the answer is the last word (edge case)
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [28]:
features = prepare_train_features(training_data[:5])


In [16]:
tokenized_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=training_data.column_names)

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [122]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 26
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 6
    })
})

In [17]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
args = TrainingArguments(
    "transformer_qa_with_batch",
    evaluation_strategy = "epoch",
    learning_rate=2e-5, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3, 
    weight_decay=0.01,
)

In [19]:
from transformers import default_data_collator

data_collator = default_data_collator

In [20]:
pushing_args = TrainingArguments(
    output_dir= "transformer_qa_with_batch",
    evaluation_strategy = "epoch",
    learning_rate=2e-5, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3, 
    weight_decay=0.01,
    push_to_hub= True
)


In [23]:
trainer = Trainer(
    model,
    pushing_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [24]:
trainer.train()

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 5.836179256439209, 'eval_runtime': 4.3529, 'eval_samples_per_second': 5.514, 'eval_steps_per_second': 0.459, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 5.672811985015869, 'eval_runtime': 4.4191, 'eval_samples_per_second': 5.431, 'eval_steps_per_second': 0.453, 'epoch': 2.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 5.591075897216797, 'eval_runtime': 5.7212, 'eval_samples_per_second': 4.195, 'eval_steps_per_second': 0.35, 'epoch': 3.0}
{'train_runtime': 163.073, 'train_samples_per_second': 1.766, 'train_steps_per_second': 0.11, 'train_loss': 5.717903984917535, 'epoch': 3.0}


TrainOutput(global_step=18, training_loss=5.717903984917535, metrics={'train_runtime': 163.073, 'train_samples_per_second': 1.766, 'train_steps_per_second': 0.11, 'train_loss': 5.717903984917535, 'epoch': 3.0})

In [25]:
trainer.push_to_hub("transformer_qa_with_batch")

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/265M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.66k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/choichoi/transformer_qa_with_batch/commit/b8cd893ee9bde943b9d2aa29c78f9e18110bdc14', commit_message='transformer_qa_with_batch', commit_description='', oid='b8cd893ee9bde943b9d2aa29c78f9e18110bdc14', pr_url=None, pr_revision=None, pr_num=None)

In [26]:

trainer.save_model(f"transformer_qa_with_batch")
torch.save(model.state_dict(),f"transformer_qa_with_batch/model.bin")
# trainer.save_model("test-qa-trained")

In [ ]:
# from transformers import AutoModelForQuestionAnswering
# import torch
# model_custom = AutoModelForQuestionAnswering.from_pretrained("transformer_qa_with_batch")
# model_custom.load_state_dict(torch.load("transformer_qa_with_batch/model.bin"))

## Evaluation

In [27]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(torch.device('cpu')) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()


odict_keys(['loss', 'start_logits', 'end_logits'])

In [28]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

## Testing Answer

In [29]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [30]:
n_best_size = 20

In [31]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [43]:
f_t = prepare_validation_features(testing_data[:5])

In [32]:
validation_features = dataset["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset["validation"].column_names
)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [33]:
raw_predictions = trainer.predict(validation_features)

  0%|          | 0/2 [00:00<?, ?it/s]

In [34]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [35]:
max_answer_length = 30

In [36]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = dataset["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]


In [37]:
valid_answers

[{'score': 1.1055446, 'text': 'Author'},
 {'score': 1.0915706, 'text': 'Abstract'},
 {'score': 1.0342126,
  'text': 'Author: Collaer ML, Hindmarsh PC, Pasterski V, Fane BA, Hines M\n Abstract'},
 {'score': 0.9810459,
  'text': 'Title:Reduced short term memory in congenital adrenal hyperplasia (CAH) and its relationship to spatial and quantitative performance.\nAuthor'},
 {'score': 0.93660593,
  'text': 'term memory in congenital adrenal hyperplasia (CAH) and its relationship to spatial and quantitative performance.\nAuthor'},
 {'score': 0.9266681,
  'text': 'plasia (CAH) and its relationship to spatial and quantitative performance.\nAuthor'},
 {'score': 0.9163219,
  'text': 'memory in congenital adrenal hyperplasia (CAH) and its relationship to spatial and quantitative performance.\nAuthor'},
 {'score': 0.9044931, 'text': 'Abstract:'},
 {'score': 0.8851044,
  'text': 'Abstract:Girls and women with classical congenital adrenal hyperplasia ('},
 {'score': 0.8693588,
  'text': 'Author: Co

In [38]:
dataset["validation"][0]['question']

'How are individuals with congenital adrenal hyperplasia (CAH) compared to controls in terms of spatial perception and quantitative abilities?'

In [39]:
dataset["validation"][0]["answer"]

"{'text': ['Individuals with congenital adrenal hyperplasia (CAH) scored lower on spatial perception and a quantitative composite compared to controls.'], 'answer_start': ['881']}"

In [40]:
dataset["validation"][0]["context"]

'Title:Reduced short term memory in congenital adrenal hyperplasia (CAH) and its relationship to spatial and quantitative performance.\nAuthor: Collaer ML, Hindmarsh PC, Pasterski V, Fane BA, Hines M\n Abstract:Girls and women with classical congenital adrenal hyperplasia (CAH) experience elevated androgens prenatally and show increased male-typical development for certain behaviors. Further, individuals with CAH receive glucocorticoid (GC) treatment postnatally, and this GC treatment could have negative cognitive consequences. We investigated two alternative hypotheses, that: (a) early androgen exposure in females with CAH masculinizes (improves) spatial perception and quantitative abilities at which males typically outperform females, or (b) CAH is associated with performance decrements in these domains, perhaps due to reduced short-term-memory (STM). Adolescent and adult individuals with CAH (40 female and 29 male) were compared with relative controls (29 female and 30 male) on spat

In [198]:
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained("choichoi/transformer_qa_with_batch")

In [208]:
# import torch
# testing_data = prepare_train_features(dataset['validation'])
# testing_data_m = {k: torch.tensor(v) for k, v in testing_data.items()}
# with torch.no_grad():
#     output = model(**testing_data_m)
# output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [54]:
from transformers import pipeline
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)


In [63]:
dataset['validation'][0]['context']

'Title:Reduced short term memory in congenital adrenal hyperplasia (CAH) and its relationship to spatial and quantitative performance.\nAuthor: Collaer ML, Hindmarsh PC, Pasterski V, Fane BA, Hines M\n Abstract:Girls and women with classical congenital adrenal hyperplasia (CAH) experience elevated androgens prenatally and show increased male-typical development for certain behaviors. Further, individuals with CAH receive glucocorticoid (GC) treatment postnatally, and this GC treatment could have negative cognitive consequences. We investigated two alternative hypotheses, that: (a) early androgen exposure in females with CAH masculinizes (improves) spatial perception and quantitative abilities at which males typically outperform females, or (b) CAH is associated with performance decrements in these domains, perhaps due to reduced short-term-memory (STM). Adolescent and adult individuals with CAH (40 female and 29 male) were compared with relative controls (29 female and 30 male) on spat

In [65]:
dataset['validation'][0]['article_title']

'Reduced short term memory in congenital adrenal hyperplasia (CAH) and its relationship to spatial and quantitative performance.'

In [75]:
context = dataset['validation'][3]['context']
result = pipe({"question":dataset['validation'][3]['question'], 'context': context}, function_to_apply="none")

In [76]:
dataset['validation'][3]['question']

'How many children were involved in the study on smart-tablet-based neurofeedback training?'

In [77]:
dataset['validation'][3]['answer']

"{'text': ['Forty children'], 'answer_start': ['55']}"

In [78]:
result

{'score': 4.407166852615774e-05,
 'start': 1135,
 'end': 1154,
 'answer': 'study suggests that'}

## Evaluation with Bleu, Rouge, Bestscore

In [79]:
import evaluate
### your code ###
bleu = evaluate.load('bleu')
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")
### your code ###

In [82]:
predict_ans = []
for i in dataset['train']:
    context = i['context']
    result = pipe({"question":i['question'], 'context': context}, function_to_apply="none")
    predict_ans.append(result['answer'])

In [83]:
predict_ans

['Autism Spectrum Disorder Symptoms.\nAuthor',
 'Attention Problems.\nAuthor',
 'RESULTS: Examination of a 100 patients showed a significant',
 'RESULTS: Five polymorphisms were associated with',
 'study supports the',
 'RESULTS: In',
 'study.\nAuthor',
 'Overview and Some Future Directions.\nAuthor',
 'Autism Spectrum Disorder Symptoms.\nAuthor',
 'Review of Intelligent Driving Style Analysis Systems and Related Artificial Intelligence Algorithms.\nAuthor',
 'RESULTS: A number of',
 'RESULTS: Five polymorphisms were associated with',
 'examine the impact of the',
 'study.\nAuthor',
 'investigation is to evaluate the possibilities for unique driver identification utilizing the approaches identified in',
 'Article',
 'examine the impact of the',
 'Challenges in',
 'report briefly about one of the most important projects in',
 'Attention Problems.\nAuthor',
 'RESULTS: A number of',
 "explores the early history of Herbert Simon's principle of",
 'RESULTS: A number of',
 'Overview and Some

In [90]:
ground_truth = []
for j in dataset['train']['answer']:
    answers = ast.literal_eval(j)
    ground_truth.append(answers['text'][0])


In [91]:
ground_truth

['Padilla N, Eklof E, Martensson GE, Bolte S, Lagercrantz H, Aden U',
 'Yes, the improvements remained through the follow-up assessment',
 'a P value </= 0.05',
 'Five polymorphisms',
 "['Olsen JP', 'Fellows RP', 'Rivera-Mindt M', 'Morgello S', 'Byrd DA']",
 'The children were assessed for ADHD symptoms in grades 1-5.',
 'addition of glue on the boxes and the vials, dilution of genuine stolen goods with water, introduction of a different active ingredient in a vial',
 'significant advances made in the emerging field of nature-inspired computing (NIC) with a focus on the physics- and biology-based approaches and algorithms',
 'Poor Brain Growth in Extremely Preterm Neonates Long Before the Onset of Autism Spectrum Disorder Symptoms.',
 'Fuzzy Logic inference systems, Hidden Markov Models and Support Vector Machines',
 "the concepts of recovery and social inclusion in mental health nurse education curricula at Master's level",
 'crucially involved in the physiology of the prefrontal cort

In [92]:
bleu_simple = bleu.compute(predictions=predict_ans, references=ground_truth)


In [93]:
bleu_simple

{'bleu': 0.021234976208991882,
 'precisions': [0.09269662921348315,
  0.04778156996587031,
  0.03361344537815126,
  0.031746031746031744],
 'brevity_penalty': 0.45542835928760594,
 'length_ratio': 0.559748427672956,
 'translation_length': 356,
 'reference_length': 636}

In [94]:
rouge_simple = rouge.compute(predictions=predict_ans,references=ground_truth)
rouge_simple

{'rouge1': 0.07121220296386915,
 'rouge2': 0.03434138253747762,
 'rougeL': 0.06965937247015196,
 'rougeLsum': 0.06946379441308448}

In [ ]:
import numpy as np

# Locally not work
bertscore_simple = bertscore.compute(predictions=predict_ans, references=ground_truth, lang="en")
bertscore_simple_averaged={}
for key in bertscore_simple.keys():
  if key!='hashcode':
    bertscore_simple_averaged[key]=np.mean(bertscore_simple[key])

### your code ###
print("Simple system:")
print(bertscore_simple_averaged)

## continue treaining


In [1]:
answer_all = True
# load Bert
model_checkpoint = "distilbert-base-uncased"

batch_size = 16


In [2]:
# qa pairs
import pandas as pd
from datasets import Dataset, DatasetDict
with open('../testing_QA/reformatting_testing_qa_extracted.csv','r',encoding='utf-8') as f:
    df_c = pd.read_csv(f)


In [3]:
df_c

,question,answer,start_point,article_title,PMID,author,context,article_date
0,Who are the authors of the article on the disc...,"Nader AM', 'Jelenic P', 'Soulieres I'","Author: ['Nader AM', 'Jelenic P', 'Soulieres I']",Discrepancy between WISC-III and WISC-IV Cogni...,26673881,"Nader AM, Jelenic P, Soulieres I",The cognitive profile and measured intellectua...,2015/12/18
1,What is the focus of the cognitive profile exa...,Examine the cognitive profile on the Wechsler ...,Abstract:The cognitive profile and measured in...,Discrepancy between WISC-III and WISC-IV Cogni...,26673881,"Nader AM, Jelenic P, Soulieres I",The cognitive profile and measured intellectua...,2015/12/18
2,How many children with autism were part of the...,51,The third aim was to examine the impact of the...,Discrepancy between WISC-III and WISC-IV Cogni...,26673881,"Nader AM, Jelenic P, Soulieres I",The cognitive profile and measured intellectua...,2015/12/18
3,What percentage increase was observed in the d...,nearly tripled,Autistic children scored significantly higher ...,Discrepancy between WISC-III and WISC-IV Cogni...,26673881,"Nader AM, Jelenic P, Soulieres I",The cognitive profile and measured intellectua...,2015/12/18
4,What are the two main perspectives focused on ...,Two different perspectives are the main focus ...,69,Systems Medicine in Oncology: Signaling Networ...,26677185,"Parodi S, Riccardi G, Castagnino N, Tortolina ...",Two different perspectives are the main focus ...,2015/12/18
...,...,...,...,...,...,...,...,...
74,What signs of manipulation were found in the c...,"addition of glue on the boxes and the vials, d...",1171,Counterfeit analysis strategy illustrated by a...,26692552,"Degardin K, Roggo Y",Medicine counterfeiting is a current problem t...,2015/12/23
75,Who are the authors of the paper on nature-ins...,"Siddique N', 'Adeli H","Author: ['Siddique N', 'Adeli H']",Nature Inspired Computing: An Overview and Som...,26693257,"Siddique N, Adeli H",This paper presents an overview of significant...,2015/12/23
76,What is the focus of the paper on nature-inspi...,significant advances made in the emerging fiel...,This paper presents an overview of significant...,Nature Inspired Computing: An Overview and Som...,26693257,"Siddique N, Adeli H",This paper presents an overview of significant...,2015/12/23
77,What has paralleled the development of 'nature...,the emergence of the field of computational in...,A parallel development in the past two decades...,Nature Inspired Computing: An Overview and Som...,26693257,"Siddique N, Adeli H",This paper presents an overview of significant...,2015/12/23


In [4]:
from copy import deepcopy
df_latest = deepcopy(df_c)

col_name=df_latest.columns.tolist() 
col_name.insert(0, 'id')

df_latest = df_latest.reindex(columns=col_name)  

for i in range(len(df_c['answer'])):
    new_context = f"Title:{df_c['article_title'][i]}\nAuthor: {df_c['author'][i]}\n Abstract:{df_c['context'][i]}\n Article date:{df_c['article_date'][i]}"
    if not df_c['start_point'][i].isnumeric():
        start_point = new_context.index(' '.join(df_c['start_point'][i].replace('[','').replace(']','').replace('\'','').split())[0:2])
        df_latest.replace({"start_point": {
                df_c['start_point'][i]: str([start_point])
            }}, inplace=True)
        temp_answer = {'text': [df_c['answer'][i]], 'answer_start': [start_point]}
    else:
        temp_answer = {'text': [df_c['answer'][i]], 'answer_start': [df_c['start_point'][i]]}
    df_latest.replace({"answer": {
                df_c['answer'][i]: str(temp_answer)
            }}, inplace=True)

    df_latest.replace({"context": {
                df_c['context'][i]: new_context
            }}, inplace=True)
    
    df_latest.at[i, 'id']= str(int(i))

C:\Users\Choi\AppData\Local\Temp\ipykernel_9252\2352966882.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_latest.at[i, 'id']= str(int(i))


In [5]:
# new data
training_df = df_latest.sample(frac=0.8, axis=0, replace=False, random_state=0)
testing_df = df_latest[~df_latest.index.isin(training_df.index)]


In [6]:
training_data = Dataset.from_pandas(training_df, preserve_index=False)
testing_data = Dataset.from_pandas(testing_df, preserve_index=False)

In [7]:
dataset = DatasetDict()
dataset['train'] = training_data
dataset['validation'] = testing_data

In [53]:
from transformers import AutoTokenizer
    

tokenizer = AutoTokenizer.from_pretrained("choichoi/transformer_qa_with_batch")

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

f:\Heidelberg\semester_5\INLPT\Final Project\.venv\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Choi\.cache\huggingface\hub\models--choichoi--transformer_qa_with_batch. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [27]:
from transformers import TrainingArguments
pushing_args = TrainingArguments(
    output_dir= "transformer_qa_with_batch",
    evaluation_strategy = "epoch",
    learning_rate=2e-5, 
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3, 
    weight_decay=0.01,
    # push_to_hub= True
)

In [33]:
pad_on_right = tokenizer.padding_side == "right" #context on the right

In [35]:
max_length = 384 # max length of input feature
doc_stride = 128 # repeat tokens number between 2 slices

In [37]:
import ast

tokenized_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=training_data.column_names)

Map:   0%|          | 0/63 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [28]:
from transformers import default_data_collator, Trainer

data_collator = default_data_collator

In [ ]:
from transformers import AutoModelForQuestionAnswering
pre_model = AutoModelForQuestionAnswering.from_pretrained("choichoi/transformer_qa_with_batch")

In [38]:
trainer = Trainer(
    pre_model,
    pushing_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

NameError: name 'Trainer' is not defined

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub("transformer_qa_with_batch")